# Dogs vs Cats Redux

In [1]:
%matplotlib inline

In [2]:
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [3]:
path = 'data/data/redux/'

In [4]:
batch_size = 32

# Todo : 
1. create validation set and sample
2. move to separate dirs for each set
3. finetune and train
4. submit

# Create validation set and sample

In [ ]:
%cd ../data/redux

In [ ]:
%cd train

In [ ]:
%mkdir ../valid

In [ ]:
!pwd

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], '../valid/' + shuf[i])

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], '../sample/valid/' + shuf[i])

# Move to separate dirs for each set

In [ ]:
%cd ../train

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../valid

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../sample/train/

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../valid

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

# Finetune and train 

In [5]:
from vgg16 import Vgg16

In [6]:
vgg = Vgg16()

/home/garvit/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [7]:
batches = vgg.get_batches(path + '/train/', batch_size = batch_size)
val_batches = vgg.get_batches(path + '/valid/', batch_size = batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch = 1)

In [ ]:
vgg.model.save_weights(path + 'results/ft1.h5')

# Submit

In [ ]:
batches, preds = vgg.test(path + 'test/', batch_size = batch_size * 2)

In [ ]:
??vgg.test()

In [ ]:
filenames = batches.filenames

In [ ]:
preds[:5]

In [ ]:
filenames[:5]

In [ ]:
save_array(path+'results/test_preds.dat', preds)
save_array(path+'results/filenames.dat', filenames)

In [ ]:
preds = load_array(path + 'results/test_preds.dat')
filenames = load_array(path + 'results/filenames.dat')

In [ ]:
9ir=from PIL import Image
Image.open(path + 'test/' + filenames[0])

In [ ]:
isdog = preds[:, 1]

In [ ]:
isdog[:5]

In [ ]:
ids = [int(f[8:f.find('.')]) for f in filenames]
ids[:5]

In [ ]:
subm = np.stack([ids, isdog], axis = 1)
subm[:5]

In [ ]:
np.savetxt('data/redux/subm98.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
from IPython.display import FileLink
FileLink('data/redux/subm98.csv')

In [ ]:
isdog = np.clip(preds[:, 1], 0.05, 0.95)

In [ ]:
isdog[:]

# Run a few more epochs....

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
vgg.model.save_weights(path+'results/ft2.h5')

In [ ]:
vgg.model.optimizer.lr = 0.01

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
vgg.model.save_weights(path+'results/ft2_1.h5')

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
vgg.model.save_weights(path+'results/ft2_2.h5')

In [ ]:
vgg.model.load_weights(path+'results/ft2_2.h5')

Retraning layers


In [ ]:
vgg.model.summary()

In [8]:
vgg.model.pop()

In [9]:
for layer in vgg.model.layers:
    layer.trainable=False

In [10]:
vgg.model.add(Dense(2,activation='softmax'))

In [11]:
opt=RMSprop(lr=0.01)

In [12]:
vgg.model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch = 1)


In [ ]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_val_samples=val_batches.nb_sample, validation_data=val_batches, nb_epoch=2)

Epoch 1/2
23000/23000 [==============================] - 1013s - loss: 0.3991 - acc: 0.9676 - val_loss: 0.2899 - val_acc: 0.9790
Epoch 2/2
22976/23000 [============================>.] - ETA: 0s - loss: 0.3494 - acc: 0.9752